In [ ]:
import json
import requests
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import boto3
from typing import Dict, List


#1 DONE get t-2 months one day's taxi data
#2 DONE get t-2 monts one day's weather data
#3 DONE upload to s3(raw_data / weather_data and raw_data / taxi_data) to_processed
#4.DONE create functions - organise code
#5. DONE CREATING A TRIGGER

def get_taxi_data(formatted_datetime: str)-> Dict:
    """
    Retrives taxi_data for the given date.
    
    Parameters:
        formatted_datetime (str): The date in "YYYY-MM-DD" format.
        
    Returns:
        Dict: A dictionary containing the taxi_data as a JSON.
    """
    taxi_url = f"https://data.cityofchicago.org/resource/ajtu-isnz.json?$where=trip_start_timestamp >= '{formatted_datetime}T00:00:00' AND trip_start_timestamp <= '{formatted_datetime}T23:59:59'&$limit=30000"
    headers = {"X-App-Token":os.environ.get("CHICHAGO_KEY")}
    
    response_taxi = requests.get(taxi_url)
    taxi_data = response_taxi.json()
    
    return taxi_data

def get_weather_data(formatted_datetime: str)-> Dict:
    """
    Retrieves weather data from the Open Meteo API for a specified date and time.

    Args:
        formatted_datetime (str): A string representing the date and time in the format "YYYY-MM-DD".

    Returns:
        Dict: A dictionary containing weather data including temperature at 2 meters (in Celsius),
              wind speed at 10 meters (in meters per second), rain (boolean indicating if it's raining), and
              precipitation (amount of precipitation in millimeters) for the specified date and time.

    Example:
         weather_data = get_weather_data("2024-05-07 12:00:00")
    """
    weather_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude":41.85,
        "longitude": -87.65,
        "start_date": formatted_datetime,
        "end_date": formatted_datetime,
        "hourly": "temperature_2m,wind_speed_10m,rain,precipitation"
    }
    response_weather = requests.get(weather_url,params=params)
    weather_data = response_weather.json()    
    
    return weather_data

def upload_to_s3(data: Dict, folder_name: str, filename: str)-> None:
    """
    Uploads data to an Amazon S3 bucket.

    Args:
        data (Dict): A dictionary containing the data to be uploaded.
        folder_name (str): The name of the folder within the S3 bucket where the data will be stored.
        filename (str): The name of the file to be created in the specified folder.

    Returns:
        None

    Example:
        data = {"key": "value"}
        upload_to_s3(data, "my_folder", "data.json")
    """
    client = boto3.client("s3")
    client.put_object(
        Bucket = "cubix-chichago-taxi-ta",
        Key = f"raw_data/to_processed/{folder_name}/{filename}",
        Body = json.dumps(data)
        )
        
def lambda_handler(event, context):
    current_datetime = datetime.now() - relativedelta(months=2)
    formatted_datetime = current_datetime.strftime("%Y-%m-%d")
    
    taxi_data_api_call = get_taxi_data(formatted_datetime) #return taxi_data
    weather_data_api_call = get_weather_data(formatted_datetime)
    taxi_filename = f"taxi_raw_{formatted_datetime}.json"
    weather_filename = f"weather_raw_{formatted_datetime}.json"
    
    
    upload_to_s3(data = taxi_data_api_call, filename=taxi_filename, folder_name="taxi_data")
    print("Taxi data has been uploaded!")
    upload_to_s3(data = weather_data_api_call, filename=weather_filename, folder_name="weather_data")
    print("Weather data has been uploaded!")




        